# 01 - Battery Data Retrieval from InfluxDB

This notebook retrieves battery monitoring data from the InfluxDB database and saves it locally for further processing. The data includes metrics such as battery temperature, voltage, current, state of charge, and other operational parameters.


## Install Required Dependencies

Install the necessary Python packages for working with InfluxDB and data manipulation.


In [ ]:
!pip install pandas influxdb-client

## Configure InfluxDB Connection

Set up the connection parameters to connect to the InfluxDB instance where battery data is stored.


In [ ]:
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS
import pandas as pd

INFLUXDB_URL = "https://influx-db-microshift-001.apps.cluster-xqdb9.dynamic.redhatworkshops.io/" # Replace with your InfluxDB instance URL
INFLUXDB_TOKEN = "admin_token"
INFLUXDB_ORG = "redhat"
INFLUXDB_BUCKET = "bms"

## Connect to InfluxDB and Define Data Retrieval Function

Establish the connection to InfluxDB and create a function to query battery data from the last hour.


In [ ]:
client = InfluxDBClient(url=INFLUXDB_URL, token=INFLUXDB_TOKEN, org=INFLUXDB_ORG)
write_api = client.write_api(write_options=SYNCHRONOUS)


def retrieve_battery_data():
    query = f'''
    from(bucket: "{INFLUXDB_BUCKET}")
      |> range(start: -1h)
      |> filter(fn: (r) => r["_measurement"] == "battery_data")
    '''
    query_api = client.query_api()
    tables = query_api.query(query, org=INFLUXDB_ORG)

    data = []
    for table in tables:
        for record in table.records:
            data.append(record.values)

    df = pd.DataFrame(data)
    return df

## Retrieve and Save Battery Data

Execute the query to retrieve battery data, clean up unnecessary columns, and save the raw data to a CSV file.


In [ ]:
df = retrieve_battery_data()
df = df.drop(columns=["result", "table", "_measurement"])
df.to_csv("./data/battery_raw.csv", index=False)

## Preview Retrieved Data

Display the first few rows and basic information about the retrieved battery data to verify the retrieval was successful.


In [ ]:
print(f"Retrieved {len(df)} rows of battery data")
df.head(10)